In [ ]:
# Install BERTopic (agak lama installnya)
!pip install bertopic pandas

from bertopic import BERTopic
import pandas as pd

In [ ]:
def get_raw_url(github_url):
    if 'github.com' in github_url and '/blob/' in github_url:
        return github_url.replace('github.com', 'raw.githubusercontent.com').replace('/blob/', '/')
    return github_url

# === CONFIG URL ===
url_file_github = "https://github.com/USERNAME_KAMU/NAMA_REPO/blob/main/data.csv"

try:
    print("⏳ Loading data...")
    df = pd.read_csv(get_raw_url(url_file_github))
    col_text = next((c for c in df.columns if c.lower() in ['text', 'tweet', 'content', 'review']), df.columns[0])
    
    docs = df[col_text].astype(str).tolist()

    # 1. Training BERTopic
    # language="multilingual" agar support bahasa Indonesia
    print("⏳ Training BERTopic (Disarankan mengaktifkan GPU di Colab)...")
    topic_model = BERTopic(language="multilingual", verbose=True)
    
    topics, probs = topic_model.fit_transform(docs)

    # 2. Tampilkan Info Topik
    print("\n=== INFO TOPIK YANG DITEMUKAN ===")
    freq = topic_model.get_topic_info()
    print(freq.head(10)) # Tampilkan 10 topik teratas

    # 3. Tampilkan detail kata kunci topik 0 (terbanyak)
    print("\nKata kunci Topik 0:")
    print(topic_model.get_topic(0))

except Exception as e:
    print(f"Error: {e}")